In [23]:
import os
import json
import earthpy as et
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import geopandas as gpd
from geopandas import GeoDataFrame
from shapely.geometry import Point
import IPython.display as disp
%matplotlib inline

# change the global options that Geopandas inherits from
pd.set_option('display.max_columns',None)

In [2]:
# Change directory to landslide-detect data path
data_path = os.path.join(et.io.HOME, "earth-analytics", "landslide-detect")
if os.path.exists(data_path):
    os.chdir(data_path)
else:
    os.makedirs(data_path)
    print('The new directory is created!')
    os.chdir(data_path)

print('Current working directory is set to: ', os.getcwd())

Current working directory is set to:  /Users/leahmanak/earth-analytics/landslide-detect


In [55]:
def create_dataframe(data_file):
    """
    Reads a csv file into a GeoPandas DataFrame. Sets the index to the 'slide.id'
    and sorts the geoDataFrame by the new index. Converts the date+time column to
    a DateTime object and drops the time. Converts the reported geocoordinates to
    a geoPandas geometry point.

    Parameters
    ----------
    data_file : str

    Returns
    -------
    dataframe
        sorted by index
    """

    gdf = gpd.read_file(data_file, index_col='slide.id').sort_values(by='slide.id')
    gdf = gdf.set_index('slide.id')
    # Convert to DatTime object, drop time
#     gdf['slide.date'] = pd.to_datetime(gdf['slide.date'])
#     gdf['slide.date'] = gdf['slide.date'].dt.date
    # Convert reported geocoordinates to geometery
    gdf[['lon', 'lat']] = gdf[['lon', 'lat']].astype(float)
    geometry = [Point(xy) for xy in zip(gdf.lon, gdf.lat)]
    gdf = gdf.drop(['lon', 'lat'], axis=1)
    gdf = GeoDataFrame(gdf, crs="EPSG:4326", geometry=geometry)

    return gdf

In [5]:
def image_search(gdf):
    """
    Iterates through a geoDataFrame and searches Google Earth Engine
    based on certain parameters within the geoDataFrame. Buffers central
    geometery search point by 1000 meters, event date by 180 days reciprocal. 

    
    Parameters
    ----------
    data_file : geodataframe

    Returns
    -------
    dataframe
        appended with imagery timestamplist
    
    """

    # Set empty list
    results = []
    
    BASE_DATE = ee.Date(gdf['slide.date'])
    
    # Data search in Google Earth Engine
    im_coll = (ee.ImageCollection('COPERNICUS/S1_GRD_FLOAT')
               .filterBounds(gdf['geometry'])
               .filter(ee.Geometry.Point.buffer({'distance': 1000}))
               .filterDate(BASE_DATE, 
                           BASE_DATE.advance(-180, 'days'), 
                           BASE_DATE.advance(180, 'days'))
#                .filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))
               .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
               .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))
               .map(lambda img: img.set('date', ee.Date(img.date()).format('YYYYMMdd')))
               .sort('date'))
    timestamplist = (im_coll.aggregate_array('date')
                            .map(lambda d: ee.String('T').cat(ee.String(d)))
                            .getInfo())
    
    # Add new column to geopandas DataFrame for avaiable data
    for value in timestamplist:
        if value == True:
            result.append(timestamplist)
        else:
            result.append('No imagery available')

    return geodataframe

In [4]:
# for id, location, point in zip(landslide_df.index,
#                                landslide_df['location'],
#                                landslide_df['point'],
#                                ):
#     print(id, location, point)

In [53]:
for row in landslide_gdf.iterrows():
    print(landslide_gdf['geometry'])

slide.id
9734    POINT (-121.43238 35.86563)
9806    POINT (-108.18528 43.51364)
Name: geometry, dtype: geometry
slide.id
9734    POINT (-121.43238 35.86563)
9806    POINT (-108.18528 43.51364)
Name: geometry, dtype: geometry


In [56]:
landslide_gdf = create_dataframe('landslides.verified.copy.csv')
landslide_gdf.head()

,slide.date,location,type,trigger,size,location_accuracy,event_title,admin_division_name,ge.lat,ge.lon,is.exact,slide.index,geometry
slide.id,,,,,,,,,,,,,
9734,2017-05-20T13:34:00Z,Big Sur Landslide,landslide,rain,very_large,exact,Big Sur Landslide,NA,35.86562803,-121.4323838,TRUE,54,POINT (-121.43238 35.86563)
9806,2017-05-19T20:14:00Z,"Rockslide on Highway 20, WY",rock_fall,rain,medium,1km,"Rockslide on Highway 20, WY",NA,NA,NA,FALSE,55,POINT (-108.18528 43.51364)
